<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/BilevelProgramming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bilevel programming in Python

This notebook presents two different strategies to solve the generic linear bilevel problem formulated below. 

```
   min_x   A*x + B*y
    s.t.   C*x <= D
           x >= 0
           min_y E*y
           s.t.  F*y <= g  
                 H*x + I*y <= J
                 y >= 0
```

The first methodology is based on the regularization of the complementarity conditions and is solved iteratively. The second methodology uses large enough constaints. More details can be found in the paper below.

Pineda, S., Bylling, H. & Morales, J.M. Efficiently solving linear bilevel programming problems using off-the-shelf optimization software. Optim Eng 19, 187–211 (2018). ([link](https://link.springer.com/article/10.1007/s11081-017-9369-y?shared-article-renderer))

## Requirements

In [ ]:
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc
!apt-get install -y -qq glpk-utils
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
import random
import pyomo.environ as pe
ipopt = pe.SolverFactory('ipopt', executable='/content/ipopt')
cbc = pe.SolverFactory('cbc', executable='/usr/bin/cbc')
glpk = pe.SolverFactory('glpk', executable='/usr/bin/glpsol')

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 161020 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

## Input data

In [ ]:
nvar = 10
ncon = 5
A = [round(abs(random.gauss(0,1)),2) for i in range(nvar)]
B = [round(abs(random.gauss(0,1)),2) for i in range(nvar)]
C = [[round(random.gauss(0,1),2) for i in range(nvar)] for j in range(ncon)]
D = [round(random.gauss(0,1),2) for j in range(ncon)]
E = [round(abs(random.gauss(0,1)),2) for i in range(nvar)]
F = [[round(random.gauss(0,1),2) for i in range(nvar)] for j in range(ncon)]
G = [round(random.gauss(0,1),2) for j in range(ncon)]
H = [[round(random.gauss(0,1),2) for i in range(nvar)] for j in range(ncon)]
I = [[round(random.gauss(0,1),2) for i in range(nvar)] for j in range(ncon)]
J = [round(random.gauss(0,1),2) for j in range(ncon)]

## Lower-level optimization problem

In [ ]:
def solve_ll(vector_x):
    m = pe.ConcreteModel()
    # Sets
    m.i = pe.Set(initialize=range(nvar),ordered=True)
    m.j = pe.Set(initialize=range(ncon),ordered=True)
    # Variables
    m.y = pe.Var(m.i,within=pe.NonNegativeReals)
    # Objective function
    def obj_rule(m):
      return sum(E[i]*m.y[i] for i in m.i)
    m.obj = pe.Objective(rule=obj_rule)
    # Constraints
    def con1_rule(m,j):
      return sum(F[j][i]*m.y[i] for i in m.i) <= G[j]
    m.con1 = pe.Constraint(m.j,rule=con1_rule)
    def con2_rule(m,j):
      return sum(H[j][i]*vector_x[i] for i in m.i) + sum(I[j][i]*m.y[i] for i in m.i) <= J[j]
    m.con2 = pe.Constraint(m.j,rule=con2_rule)
    # Solve the lower level problem
    glpk.solve(m).write()
    # Returns the objective value of the bilevel problem
    return sum(A[i]*vector_x[i] + B[i]*m.y[i].value for i in m.i)   

## Solving the bilevel problem using regularization

In [ ]:
# Values of epsilon
vector_ep = [10**6,10**4,10**2,1,0.1,0.01,0]
# Model
m = pe.ConcreteModel()
# Sets
m.i = pe.Set(initialize=range(nvar),ordered=True)
m.j = pe.Set(initialize=range(ncon),ordered=True)
# Parameters
m.ep = pe.Param(initialize=10**6,mutable=True)
# Variables
m.x = pe.Var(m.i,within=pe.NonNegativeReals)
m.y = pe.Var(m.i,within=pe.NonNegativeReals)
m.al = pe.Var(m.j,within=pe.NonNegativeReals)
m.be = pe.Var(m.j,within=pe.NonNegativeReals)
m.ga = pe.Var(m.i,within=pe.NonNegativeReals)
# Objective function
def obj_rule(m):
  return sum(A[i]*m.x[i] for i in m.i) + sum(B[i]*m.y[i] for i in m.i)
m.obj = pe.Objective(rule=obj_rule)
# Constraints
def con1_rule(m,j):
  return sum(C[j][i]*m.x[i] for i in m.i) <= D[j]
m.con1 = pe.Constraint(m.j,rule=con1_rule)
def con2_rule(m,j):
  return sum(F[j][i]*m.y[i] for i in m.i) <= G[j]
m.con2 = pe.Constraint(m.j,rule=con2_rule)
def con3_rule(m,j):
  return sum(H[j][i]*m.x[i] for i in m.i) + sum(I[j][i]*m.y[i] for i in m.i) <= J[j]
m.con3 = pe.Constraint(m.j,rule=con3_rule)
def con4_rule(m,i):
  return E[i] + sum(F[j][i]*m.al[j] for j in m.j) + sum(I[j][i]*m.be[j] for j in m.j) - m.ga[i] == 0
m.con4 = pe.Constraint(m.i,rule=con4_rule)
def con5_rule(m):
  return sum((G[j] - sum(F[j][i]*m.y[i] for i in m.i))*m.al[j] for j in m.j) + sum((J[j] - sum(H[j][i]*m.x[i] for i in m.i) - sum(I[j][i]*m.y[i] for i in m.i))*m.be[j] for j in m.j) + sum(m.y[i]*m.ga[i] for i in m.i) <= m.ep
m.con5 = pe.Constraint(rule=con5_rule)
# Solve the model iteratively
for ep in vector_ep:
  m.ep = ep
  ipopt.solve(m).write()
# Output solution
x_reg = [m.x[i].value for i in m.i]
of_reg = solve_ll(x_reg)
print('Optimal solution:',x_reg)
print('Optimal value:',of_reg)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 26
  Number of variables: 40
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.025297880172729492
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
# ==

## Solving the bilevel problem using bigM

In [ ]:
# Big M
BigM = 10**6
# Model
m = pe.ConcreteModel()
# Sets
m.i = pe.Set(initialize=range(nvar),ordered=True)
m.j = pe.Set(initialize=range(ncon),ordered=True)
# Variables
m.x = pe.Var(m.i,within=pe.NonNegativeReals)
m.y = pe.Var(m.i,within=pe.NonNegativeReals)
m.al = pe.Var(m.j,within=pe.NonNegativeReals)
m.be = pe.Var(m.j,within=pe.NonNegativeReals)
m.ga = pe.Var(m.i,within=pe.NonNegativeReals)
m.u1 = pe.Var(m.j,within=pe.Binary)
m.u2 = pe.Var(m.j,within=pe.Binary)
m.u3 = pe.Var(m.i,within=pe.Binary)
# Objective function
def obj_rule(m):
  return sum(A[i]*m.x[i] for i in m.i) + sum(B[i]*m.y[i] for i in m.i)
m.obj = pe.Objective(rule=obj_rule)
# Constraints
def con1_rule(m,j):
  return sum(C[j][i]*m.x[i] for i in m.i) <= D[j]
m.con1 = pe.Constraint(m.j,rule=con1_rule)
def con2_rule(m,j):
  return sum(F[j][i]*m.y[i] for i in m.i) <= G[j]
m.con2 = pe.Constraint(m.j,rule=con2_rule)
def con3_rule(m,j):
  return sum(H[j][i]*m.x[i] for i in m.i) + sum(I[j][i]*m.y[i] for i in m.i) <= J[j]
m.con3 = pe.Constraint(m.j,rule=con3_rule)
def con4_rule(m,i):
  return E[i] + sum(F[j][i]*m.al[j] for j in m.j) + sum(I[j][i]*m.be[j] for j in m.j) - m.ga[i] == 0
m.con4 = pe.Constraint(m.i,rule=con4_rule)
def con5_rule(m,j):
  return G[j] - sum(F[j][i]*m.y[i] for i in m.i) <= m.u1[j]*BigM
m.con5 = pe.Constraint(m.j,rule=con5_rule)
def con6_rule(m,j):
  return m.al[j] <= (1-m.u1[j])*BigM
m.con6 = pe.Constraint(m.j,rule=con6_rule)
def con7_rule(m,j):
  return J[j] - sum(H[j][i]*m.x[i] for i in m.i) - sum(I[j][i]*m.y[i] for i in m.i) <= m.u2[j]*BigM
m.con7 = pe.Constraint(m.j,rule=con7_rule)
def con8_rule(m,j):
  return m.be[j] <= (1-m.u2[j])*BigM
m.con8 = pe.Constraint(m.j,rule=con8_rule)
def con9_rule(m,i):
  return m.y[i] <= m.u3[i]*BigM
m.con9 = pe.Constraint(m.i,rule=con9_rule)
def con10_rule(m,i):
  return m.ga[i] <= (1-m.u3[i])*BigM
m.con10 = pe.Constraint(m.i,rule=con10_rule)
# Solve the model
cbc.solve(m).write()
# Output solution
x_BigM = [m.x[i].value for i in m.i]
of_BigM = solve_ll(x_BigM)
print('Optimal solution:',x_BigM)
print('Optimal value:',of_BigM)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 6.1192309
  Upper bound: 6.1192309
  Number of objectives: 1
  Number of constraints: 65
  Number of variables: 60
  Number of binary variables: 20
  Number of integer variables: 20
  Number of nonzeros: 20
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.27
  Wallclock time: 0.29
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch a